# Semantic Kernel Tool Use Example

This document provides an overview and explanation of the code used to create a Semantic Kernel-based tool that integrates with ChromaDB for Retrieval-Augmented Generation (RAG). The example demonstrates how to build an AI agent that retrieves travel documents from a ChromaDB collection, augments user queries with semantic search results, and streams detailed travel recommendations.

## Initializing the Environment

SQLite Version Fix
If you encounter the error:
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```

Uncomment this code block at the start of your notebook:

In [2]:
# %pip install pysqlite3-binary
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importing Packages
The following code imports the necessary packages:

In [2]:
import os
import chromadb

from openai import AsyncOpenAI
from typing import Annotated
from IPython.display import display, HTML

from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.agents import ChatCompletionAgent

### Creating the Semantic Kernel and AI Service

A Semantic Kernel instance is created and configured with an asynchronous OpenAI chat completion service. The service is added to the kernel for use in generating responses.

In [3]:
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)

# Create a Semantic Kernel instance and add an OpenAI chat completion service.
kernel = Kernel()
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
    service_id="agent",
)
kernel.add_service(chat_completion_service)

### Defining the Prompt Plugin

The PromptPlugin is a native plugin that defines a function to build an augmented prompt using retrieval context

In [4]:
class PromptPlugin:
    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context."
    )
    @staticmethod
    def build_augmented_prompt(query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide your answer."
        )

# Register the plugin with the kernel.
kernel.add_plugin(PromptPlugin(), plugin_name="promptPlugin")

KernelPlugin(name='promptPlugin', description=None, functions={})

### Defining Weather Information Plugin

The WeatherInfoPlugin is a native plugin that provides temperature information for specific travel destinations.

In [5]:
class WeatherInfoPlugin:
    """A Plugin that provides the average temperature for a travel destination."""

    def __init__(self):
        # Dictionary of destinations and their average temperatures
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="Get the average temperature for a specific travel destination.")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "Returns the average temperature for the destination."]:
        """Get the average temperature for a travel destination."""
        # Normalize the input destination (lowercase)
        normalized_destination = destination.lower()

        # Look up the temperature for the destination
        if normalized_destination in self.destination_temperatures:
            return f"The average temperature in {destination} is {self.destination_temperatures[normalized_destination]}."
        else:
            return f"Sorry, I don't have temperature information for {destination}. Available destinations are: Maldives, Swiss Alps, and African safaris."
        
# Register the weather plugin with the kernel.
kernel.add_plugin(WeatherInfoPlugin(), plugin_name="weatherPlugin")

KernelPlugin(name='weatherPlugin', description=None, functions={'get_destination_temperature': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='get_destination_temperature', plugin_name='weatherPlugin', description='Get the average temperature for a specific travel destination.', parameters=[KernelParameterMetadata(name='destination', description=None, default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='Returns the average temperature for the destination.', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'Returns the average temperature for the destination.'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._Pro

### Defining Destinations Information Plugin

The DestinationsPlugin is a native plugin that provides detailed information about popular travel destinations.

In [6]:
class DestinationsPlugin:
    # Destination data store with rich details about popular travel locations
    DESTINATIONS = {
        "maldives": {
            "name": "The Maldives",
            "description": "An archipelago of 26 atolls in the Indian Ocean, known for pristine beaches and overwater bungalows.",
            "best_time": "November to April (dry season)",
            "activities": ["Snorkeling", "Diving", "Island hopping", "Spa retreats", "Underwater dining"],
            "avg_cost": "$400-1200 per night for luxury resorts"
        },
        "swiss alps": {
            "name": "The Swiss Alps",
            "description": "Mountain range spanning across Switzerland with picturesque villages and world-class ski resorts.",
            "best_time": "December to March for skiing, June to September for hiking",
            "activities": ["Skiing", "Snowboarding", "Hiking", "Mountain biking", "Paragliding"],
            "avg_cost": "$250-500 per night for alpine accommodations"
        },
        "safari": {
            "name": "African Safari",
            "description": "Wildlife viewing experiences across various African countries including Kenya, Tanzania, and South Africa.",
            "best_time": "June to October (dry season) for optimal wildlife viewing",
            "activities": ["Game drives", "Walking safaris", "Hot air balloon rides", "Cultural village visits"],
            "avg_cost": "$400-800 per person per day for luxury safari packages"
        },
        "bali": {
            "name": "Bali, Indonesia",
            "description": "Island paradise known for lush rice terraces, beautiful temples, and vibrant culture.",
            "best_time": "April to October (dry season)",
            "activities": ["Surfing", "Temple visits", "Rice terrace trekking", "Yoga retreats", "Beach relaxation"],
            "avg_cost": "$100-500 per night depending on accommodation type"
        },
        "santorini": {
            "name": "Santorini, Greece",
            "description": "Stunning volcanic island with white-washed buildings and blue domes overlooking the Aegean Sea.",
            "best_time": "Late April to early November",
            "activities": ["Sunset watching in Oia", "Wine tasting", "Boat tours", "Beach hopping", "Ancient ruins exploration"],
            "avg_cost": "$200-600 per night for caldera view accommodations"
        }
    }

    @kernel_function(
        name="get_destination_info",
        description="Provides detailed information about specific travel destinations."
    )
    @staticmethod
    def get_destination_info(query: str) -> str:
        # Find which destination is being asked about
        query_lower = query.lower()
        matching_destinations = []

        for key, details in DestinationsPlugin.DESTINATIONS.items():
            if key in query_lower or details["name"].lower() in query_lower:
                matching_destinations.append(details)

        if not matching_destinations:
            return (f"User Query: {query}\n\n"
                    f"I couldn't find specific destination information in our database. "
                    f"Please use the general retrieval system for this query.")

        # Format destination information
        destination_info = "\n\n".join([
            f"Destination: {dest['name']}\n"
            f"Description: {dest['description']}\n"
            f"Best time to visit: {dest['best_time']}\n"
            f"Popular activities: {', '.join(dest['activities'])}\n"
            f"Average cost: {dest['avg_cost']}" for dest in matching_destinations
        ])

        return (f"Destination Information:\n{destination_info}\n\n"
                f"User Query: {query}\n\n"
                "Based on the above destination details, provide a helpful response "
                "that addresses the user's query about this location.")

# Register the destinations plugin with the kernel.
kernel.add_plugin(DestinationsPlugin(), plugin_name="destinationsPlugin")

KernelPlugin(name='destinationsPlugin', description=None, functions={})

## Setting Up ChromaDB

To facilitate Retrieval-Augmented Generation, a persistent ChromaDB client is instantiated and a collection named `"travel_documents"` is created (or retrieved if it exists). This collection is then populated with sample travel documents and metadata.

In [7]:
collection = chromadb.PersistentClient(path="./chroma_db").create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True,
)

documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours.",
]

collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

A helper function `get_retrieval_context` is defined to query the collection and return the top two relevant documents (with metadata) based on the user query:

In [8]:
def get_retrieval_context(query: str) -> str:
    results = collection.query(
        query_texts=[query],
        include=["documents", "metadatas"],
        n_results=2
    )
    context_entries = []
    if results and results.get("documents") and results["documents"][0]:
        for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
            context_entries.append(f"Document: {doc}\nMetadata: {meta}")
    return "\n\n".join(context_entries) if context_entries else "No retrieval context found."

## Setting the Function Choice Behavior 

In Semantic Kernel, we have the ability to have some control of the agent choice of functions. This is done by using the `FunctionChoiceBehavior` class. 

The code below sets it to `Auto` which allows the agent to choose among the available functions or not choose any. 

This can also be set to:
`FunctionChoiceBehavior.Required` - to require the agent to choose at least one function 
`FunctionChoiceBehavior.NoneInvoke` - instructs the agent to not choose any function. (good for testing)

In [9]:
settings = kernel.get_prompt_execution_settings_from_service_id("agent")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
arguments = KernelArguments(settings=settings)

In [10]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = (
    "Answer travel queries using the provided context. If context is provided, do not say 'I have no context for that.'"
)
agent = ChatCompletionAgent(
    kernel=kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=arguments,
)

A helper function `get_augmented_prompt` forces a call to the plugin to build the augmented prompt. It directly calls the static plugin method:

In [11]:
async def get_augmented_prompt(query: str) -> str:
    retrieval_context = get_retrieval_context(query)
    return PromptPlugin.build_augmented_prompt(query, retrieval_context)

### Running the Agent with Streaming Chat History
The main asynchronous loop creates a chat history for the conversation and, for each user input, first adds the augmented prompt (as a system message) to the chat history so that the agent sees the retrieval context. The user message is also added, and then the agent is invoked using streaming. The output is printed as it streams in.

In [14]:
async def main():
    # Create a chat history.
    chat_history = ChatHistory()

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?", # Retrieval context available.
        "What is the average tempature of the Maldives?", # Weather plugin can help
        "What is a good cold destination offered by Conotso and what is it average temparture?", # Weather plugin can help
        "What is Neural Network?",  # No retrieval context available.
        "Tell me more about the Swiss Alps as a destination." # Destinations plugin can help
    ]

    for user_input in user_inputs:
        # Add the user message to chat history
        chat_history.add_user_message(user_input)
        augmented_prompt = await get_augmented_prompt(user_input)
        
        chat_history.add_system_message(
            f"Here is relevant information to help answer the user's question: {augmented_prompt}")


        # Display the augmented prompt in a collapsible section
        html_output = f"<div style='margin-bottom:10px'>"
        html_output += f"<details>"
        html_output += f"<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>RAG Context (click to expand)</summary>"
        html_output += f"<div style='margin:10px; padding:10px; background-color:#f8f8f8; border:1px solid #ddd; border-radius:4px; white-space:pre-wrap;'>{augmented_prompt}</div>"

        html_output += f"</details>"
        html_output += f"</div>"

        # Show user query
        html_output += f"<div style='margin-bottom:10px'>"
        html_output += f"<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += f"</div>"

        agent_name: str | None = None
        full_response = ""
        function_calls = []
        function_results = {}

        # Collect the agent's response with improved content handling
        async for content in agent.invoke_stream(chat_history):
            if not agent_name and hasattr(content, 'name'):
                agent_name = content.name

            # Track function calls and results
            for item in content.items:
                if isinstance(item, FunctionCallContent):
                    call_info = f"Calling: {item.function_name}({item.arguments})"
                    function_calls.append(call_info)
                elif isinstance(item, FunctionResultContent):
                    result_info = f"Result: {item.result}"
                    function_calls.append(result_info)
                    # Store function results to possibly add to chat history
                    function_results[item.function_name] = item.result

            # Better content extraction - make sure we're getting the actual text
            if hasattr(content, 'content') and content.content and content.content.strip():
                # Check if this is a regular text message (not function related)
                if not any(isinstance(item, (FunctionCallContent, FunctionResultContent))
                           for item in content.items):
                    full_response += content.content

        # Add function call info to chat history
        if function_results:
            # Even if we have some response text, we want to make sure function results are incorporated
            function_results_message = "Function calls completed with the following results: " + \
                str(function_results)
            chat_history.add_system_message(function_results_message)

            # Get final response from agent that incorporates the function results
            collected_response = ""
            async for content in agent.invoke_stream(chat_history):
                if hasattr(content, 'content') and content.content and content.content.strip():
                    collected_response += content.content

            if collected_response:
                full_response = collected_response

        # Add function calls to HTML if any occurred
        if function_calls:
            html_output += f"<div style='margin-bottom:10px'>"
            html_output += f"<details>"
            html_output += f"<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
            html_output += f"<div style='margin:10px; padding:10px; background-color:#f8f8f8; border:1px solid #ddd; border-radius:4px; white-space:pre-wrap;'>"
            html_output += "<br>".join(function_calls)
            html_output += f"</div></details></div>"

        # Add agent response to HTML - make sure we have a valid response
        html_output += f"<div style='margin-bottom:20px'>"
        html_output += f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
        html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{full_response}</div>"
        html_output += f"</div>"
        html_output += "<hr>"

        # Add agent's response to chat history
        if full_response:
            chat_history.add_assistant_message(full_response)

        # Display formatted HTML
        display(HTML(html_output))

await main()